In [1]:
import torch
import torch.nn as nn

In [2]:
# from models.common import *
from yolov7_utils.common import Conv, Concat, MP, SPPCSPC, RepConv
from nas.supernet.yolo import IDetect
from yolov7_utils.general import make_divisible
from copy import deepcopy

# libraries from ofa
from nas.supernet.dynamic_layers import DynamicConvLayer, DynamicConv2d, DynamicBatchNorm2d
from nas.supernet.search_block import ELAN, ELANBlock, BBoneELAN, HeadELAN, DyConv

# TODO

* ELANBlock 상위 클래스 만들어서 기본 메소드 상속하는 방식

In [3]:
# # from yolov7 in common.py
# class Conv(nn.Module):
#     # Standard convolution
#     def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
#         super(Conv, self).__init__()
#         p=k//2
#         self.conv = nn.Conv2d(c1, c2, k, s, p, groups=g, bias=False)
#         self.bn = nn.BatchNorm2d(c2)
#         self.act = nn.SiLU() if act is True else (act if isinstance(act, nn.Module) else nn.Identity())

#     def forward(self, x):
#         return self.act(self.bn(self.conv(x)))

#     def fuseforward(self, x):
#         return self.act(self.conv(x))


# class Concat(nn.Module):
#     def __init__(self, dimension=1):
#         super(Concat, self).__init__()
#         self.d = dimension

#     def forward(self, x):
#         return torch.cat(x, self.d)


# class DyConv(nn.Module):
#     # Dynamic Convolution for elastic channel size
#     def __init__(self, c1, c2, k=1, s=1, act=True):  # ch_in, ch_out, kernel, stride
#         super(DyConv, self).__init__()
#         self.conv = DynamicConv2d(c1, c2, k, s) # auto same padding
#         self.bn = DynamicBatchNorm2d(c2)
#         self.act = nn.SiLU() if act is True else (act if isinstance(act, nn.Module) else nn.Identity())

#     def forward(self, x):
#         return self.act(self.bn(self.conv(x)))

#     def fuseforward(self, x):
#         return self.act(self.conv(x))

In [4]:
class ELAN(nn.Module):
    def __init__(self, mode, c1, c2, k, depth):
        super(ELAN, self).__init__()
        assert c1 % 2 == 0
        self.c2 = c2
        self.depth = depth

In [5]:
class ELANBlock(nn.Module):
    def __init__(self, mode, layers, depth):
        super(ELANBlock, self).__init__()
        self.layers = layers
        if mode == 'BBone':
            self.act_idx = [idx for idx in range(depth * 2) if (idx % 2 == 1 or idx == 0)] # it is only for Bbone
        elif mode == 'Head':
            self.act_idx = [idx for idx in range(depth + 1)]
        else:
            raise ValueError
            
    def forward(self, x, d=None):
        outputs = []
        for i, m in enumerate(self.layers):
            if i == 0:
                outputs.append(m(x))
            else:
                x = m(x)
                outputs.append(x)
                
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)

* Backbone ELAN Block

In [14]:
input = torch.randn(1, 128, 64, 64)
block = BBoneELAN(c1=128, c2=64, k=3, depth=4)

In [15]:
block(input).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 320, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 192, 64, 64]),
 torch.Size([1, 128, 64, 64]))

* Dynamic Convolution 검증

In [16]:
# c2 * (depth + 1)
dyconv = DyConv(64*(4+1), 256, 3, 1)

In [17]:
dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [ ]:
# TODO we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 64, 64])

* Head ELAN Block

In [18]:
input = torch.randn(1, 512, 64, 64)
block = HeadELAN(c1=512, c2=256, k=3, depth=5)

In [19]:
block(input).shape, block(input, d=4).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 1024, 64, 64]),
 torch.Size([1, 896, 64, 64]),
 torch.Size([1, 768, 64, 64]),
 torch.Size([1, 640, 64, 64]),
 torch.Size([1, 512, 64, 64]))

* Dynamic Convolution 검증

In [20]:
# c2 // 2 * (depth + 3)
dyconv = DyConv(256//2*(5+3), 256, 3, 1)

In [21]:
dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [22]:
# we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 64, 64])

* get_active_subnet method 검증

In [23]:
# This part is for get_active_subnet in supernet_yolov7.py
b_block = BBoneELAN(c1=128, c2=64, k=3, depth=4)
depth = 3
act_idx = b_block.act_idx[depth]
layers = deepcopy(b_block.layers[:act_idx+1])
block = ELANBlock(b_block.mode, layers, depth)

In [24]:
input = torch.randn(1, 128, 64, 64)
block(input).shape

torch.Size([1, 256, 64, 64])

# YOLOSuperNet

* parse_supernet 함수 정의
: BBoneELAN과 HeadELAN에 대한 c2 계산 방식이 상이함.

In [25]:
def parse_supernet(d, ch):  # model_dict, input_channels(3)
    print('\n%3s%18s%3s%10s  %-40s%-30s' % ('', 'from', 'n', 'params', 'module', 'arguments'))
    anchors, nc, gd, gw = d['anchors'], d['nc'], d['depth_multiple'], d['width_multiple']
    na = (len(anchors[0]) // 2) if isinstance(anchors, list) else anchors  # number of anchors
    no = na * (nc + 5)  # number of outputs = anchors * (classes + 5)

    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):  # from, number, module, args
        m = eval(m) if isinstance(m, str) else m  # eval strings
        for j, a in enumerate(args):
            try:
                args[j] = eval(a) if isinstance(a, str) else a  # eval strings
            except:
                pass

        n = max(round(n * gd), 1) if n > 1 else n  # depth gain
        if m in [nn.Conv2d, Conv, RepConv, SPPCSPC, DyConv]:
            c1, c2 = ch[f], args[0]
            if c2 != no:  # if not output
                c2 = make_divisible(c2 * gw, 8)

            args = [c1, c2, *args[1:]]
            if m in [SPPCSPC]:
                args.insert(2, n)  # number of repeats
                n = 1
        elif m is BBoneELAN:
            c1, c2 = ch[f], int(args[0]*(args[-1]+1))
            args = [c1, *args]
        elif m is HeadELAN:
            c1, c2 = ch[f], int((args[0]*2) + (args[0]/2 * (args[-1]-1)))
            args = [c1, *args]
        elif m is nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum([ch[x] for x in f])
        # elif m is Chuncat:
        #     c2 = sum([ch[x] for x in f])
        # elif m is Shortcut:
        #     c2 = ch[f[0]]
        # elif m is Foldcut:
        #     c2 = ch[f] // 2
        elif m in [IDetect]:
            args.append([ch[x] for x in f])
            if isinstance(args[1], int):  # number of anchors
                args[1] = [list(range(args[1] * 2))] * len(f)
        # elif m is ReOrg:
        #     c2 = ch[f] * 4
        # elif m is Contract:
        #     c2 = ch[f] * args[0] ** 2
        # elif m is Expand:
        #     c2 = ch[f] // args[0] ** 2
        else:
            c2 = ch[f]
        print(m)
        m_ = nn.Sequential(*[m(*args) for _ in range(n)]) if n > 1 else m(*args)  # module
        t = str(m)[8:-2].replace('__main__.', '')  # module type
        np = sum([x.numel() for x in m_.parameters()])  # number params
        m_.i, m_.f, m_.type, m_.np = i, f, t, np  # attach index, 'from' index, type, number params
        print('%3s%18s%3s%10.0f  %-40s%-30s' % (i, f, n, np, t, args))  # print
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)  # append to savelist
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
    return nn.Sequential(*layers), sorted(save)

In [27]:
import yaml
yaml_file = 'yaml/yolov7_elan_test.yml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

In [31]:
# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

In [32]:
model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 1]                 
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2]                
<class 'yolov7_utils.common.Conv'>
  2                -1  1     36992  yolov7_utils.common.Conv                [64, 64, 3, 1]                
<class 'yolov7_utils.common.Conv'>
  3                -1  1     73984  yolov7_utils.common.Conv                [64, 128, 3, 2]               
<class 'nas.supernet.search_block.BBoneELAN'>
  4                -1  1    164608  nas.supernet.search_block.BBoneELAN     [128, 64, 3, 3]               
<class 'yolov7_utils.common.Conv'>
  5                -1  1     66048  yolov7_utils.common.Conv                [256, 256, 1, 1]              
<class 'yolov7_utils.common.M

In [33]:
from yolov7_utils.common import * 
from nas.supernet.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [34]:
input = torch.randn(1, 3, 256, 256)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

[tensor([[[[[ 2.95383e-01, -1.99193e-01, -2.37044e-01,  ...,  1.59581e-01,  8.95222e-02,  2.56679e-01],
            [ 2.01551e-01, -4.39636e-01, -2.68227e-01,  ...,  1.94344e-01,  3.54802e-01,  1.54338e-01],
            [ 2.28810e-01, -3.33057e-01, -2.64365e-01,  ...,  1.64757e-01,  4.21801e-01,  1.31127e-01],
            ...,
            [-8.18814e-02, -2.96206e-03, -2.88915e-01,  ...,  3.58020e-01,  4.21363e-01,  4.42459e-01],
            [ 2.08815e-01, -2.09474e-01, -1.60111e-01,  ...,  4.19130e-01,  2.47611e-01,  2.46729e-01],
            [ 3.72366e-01, -1.24494e-01, -3.80982e-01,  ...,  4.56977e-01, -2.20732e-02,  1.23470e-01]],
 
           [[ 2.11668e-01, -5.27433e-01, -2.08590e-01,  ...,  3.99108e-01,  1.64146e-01,  3.66251e-01],
            [ 3.48850e-01, -4.90876e-01, -3.21098e-01,  ..., -2.42844e-02,  6.43075e-02, -1.36846e-01],
            [ 5.32371e-02,  6.99125e-03,  2.07328e-02,  ...,  1.08863e-02,  8.23316e-02,  1.37348e-01],
            ...,
            [-6.92503e-03, 

## YOLOSuperNet 검증

In [36]:
import yaml
yaml_file = 'yaml/yolov7_dynamicsupernet.yml'
with open(yaml_file) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [37]:
# Define model
ch, nc, anchors = 3, None, None

ch = config['ch'] = config.get('ch', ch)  # input channels
if nc and nc != config['nc']:
    print(f"Overriding model.yaml nc={config['nc']} with nc={nc}")
    config['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    config['anchors'] = round(anchors)  # override yaml value

In [38]:
model, save = parse_supernet(deepcopy(config), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 1]                 
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2]                
<class 'yolov7_utils.common.Conv'>
  2                -1  1     36992  yolov7_utils.common.Conv                [64, 64, 3, 1]                
<class 'yolov7_utils.common.Conv'>
  3                -1  1     73984  yolov7_utils.common.Conv                [64, 128, 3, 2]               
<class 'nas.supernet.search_block.BBoneELAN'>
  4                -1  1    164608  nas.supernet.search_block.BBoneELAN     [128, 64, 3, 3]               
<class 'nas.supernet.search_block.DyConv'>
  5                -1  1     66048  nas.supernet.search_block.DyConv        [256, 256, 1, 1]              
<class 'yolov7_utils.

In [39]:
def get_active_net_config(yaml, runtime_depth): # self
    idx = 0
    d = deepcopy(yaml)
    
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
        if 'ELAN'in m:
            args[-1] = runtime_depth[idx]
            idx += 1
    
    del d['depth_list']
            
    return d
    

In [40]:
runtime_depth = [3,2,2,1,5,5,5,3]
d = get_active_net_config(config, runtime_depth)

In [41]:
for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
    if 'ELAN'in m:
        print(i, m, args)

4 BBoneELAN [64, 3, 3]
11 BBoneELAN [128, 3, 2]
18 BBoneELAN [256, 3, 2]
25 BBoneELAN [256, 3, 1]
32 HeadELAN [256, 3, 5]
38 HeadELAN [128, 3, 5]
45 HeadELAN [256, 3, 5]
52 HeadELAN [512, 3, 3]


In [43]:
from yolov7_utils.common import * 
from nas.supernet.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [44]:
input = torch.randn(1, 3, 256, 256)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

[tensor([[[[[-5.12626e-01, -2.12486e-01,  7.29461e-02,  ..., -9.46013e-02, -1.49151e-01,  3.10560e-01],
            [-5.26625e-01,  2.78568e-01,  1.46677e-01,  ...,  1.89838e-01, -3.96693e-01,  3.62671e-01],
            [ 5.63842e-02, -5.15891e-03, -1.01364e-01,  ...,  1.38700e-01, -2.22295e-01,  1.21738e-01],
            ...,
            [-4.93032e-01, -4.79689e-01, -2.84990e-01,  ..., -5.52354e-03, -9.27650e-02,  2.57976e-01],
            [-1.37420e-01,  5.98946e-02,  3.79577e-01,  ..., -1.55369e-01, -1.88062e-01,  3.04876e-01],
            [-4.11585e-01,  1.79435e-01,  1.39512e-01,  ...,  1.87518e-01, -8.37151e-03,  2.55889e-01]],
 
           [[ 3.80759e-02,  6.04044e-02,  1.64540e-01,  ..., -6.47056e-02,  7.55637e-02,  9.42520e-02],
            [-4.12101e-01, -7.54018e-02,  1.69501e-01,  ...,  8.14262e-02, -1.10328e-01, -2.06994e-02],
            [-1.28312e-01, -1.21234e-01,  1.88257e-01,  ...,  4.35028e-01, -6.61205e-01,  1.90765e-01],
            ...,
            [-5.40518e-01, 

In [45]:
x = torch.randn(1, 3, 256, 256)

y, dt = [], []  # outputs
for i, m in enumerate(model):
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
        
    x = m(x)  # run
    
    y.append(x if m.i in save else None)  # save output

In [46]:
len(x)

3

## ETC

* 초기버전 BBoneELAN, HeadELAN

In [51]:
# ELANBlock for Backbone
class BBoneELAN(ELAN):
    type = 'BBone'
    def __init__(self, c1, c2, k, depth):
        super(BBoneELAN, self).__init__(c1, c2, k, depth)
        assert c1 % 2 == 0 and depth < 5
        c_ = int(c1 / 2)
        self.c2 = c2
        
        self.depth = depth
        
        layers = []
        
        # depth 1
        self.cv1 = Conv(c1, c2, 1, 1)
        self.cv2 = Conv(c1, c2, 1, 1)
        # depth 2
        self.cv3 = Conv(c2, c2, k, 1)
        self.cv4 = Conv(c2, c2, k, 1)
        # depth 3
        self.cv5 = Conv(c2, c2, k, 1)
        self.cv6 = Conv(c2, c2, k, 1)
        # depth 4
        self.cv7 = Conv(c2, c2, k, 1)
        self.cv8 = Conv(c2, c2, k, 1)
        
        self.act_idx = [0, 1, 3, 5, 7][:depth+1] 
    
        layers.append(self.cv1)
        layers.append(self.cv2)
        layers.append(self.cv3)
        layers.append(self.cv4)
        layers.append(self.cv5)
        layers.append(self.cv6)
        layers.append(self.cv7)
        layers.append(self.cv8)
        self.layers = nn.Sequential(*layers)
    
    def set_depth(self, depth):
        self.depth = depth
        self.out_C = self.c2 * (depth + 1)
    
    def forward_once(self, x, d=None):
        outputs = []
        for i, m in enumerate(self.layers):
            if i == 0:
                outputs.append(m(x))
            else:
                x = m(x)
                outputs.append(x)
                
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)
    
    def forward(self, x, d=None):
        outputs = []
        # depth 1
        x1 = self.cv1(x)
        outputs.append(x1)
        x2 = self.cv2(x)    
        outputs.append(x2)
        # depth 2
        x3 = self.cv3(x2)
        outputs.append(x3)
        x4 = self.cv4(x3)
        outputs.append(x4)
        # depth 3
        x5 = self.cv5(x4)
        outputs.append(x5)
        x6 = self.cv6(x5)
        outputs.append(x6)
        # depth 4
        x7 = self.cv7(x6)
        outputs.append(x7)
        x8 = self.cv8(x7)
        outputs.append(x8)
        
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)
    
# ELANBlock for Head
# there are differences about cardinality(path) and channel size
class HeadELAN(ELAN):
    def __init__(self, c1, c2, k, depth):
        super(HeadELAN, self).__init__(c1, c2, k, depth)
        assert c1 % 2 == 0 and c2 % 2 == 0 and depth < 6
        c_ = int(c2 / 2)
        self.c2 = c2
        self.depth = depth
        
        # depth 1
        self.cv1 = Conv(c1, c2, 1, 1)
        self.cv2 = Conv(c1, c2, 1, 1)
        # depth 2
        self.cv3 = Conv(c2, c_, k, 1)
        # depth 3
        self.cv4 = Conv(c_, c_, k, 1)
        # depth 4
        self.cv5 = Conv(c_, c_, k, 1)
        # depth 5
        self.cv6 = Conv(c_, c_, k, 1)
        
        self.act_idx = [0, 1, 2, 3, 4, 5, 6][:depth+1] 
    
    def forward(self, x, d=None):
        outputs = []
        # depth 1
        x1 = self.cv1(x)
        outputs.append(x1)
        x2 = self.cv2(x)    
        outputs.append(x2)
        # depth 2
        x3 = self.cv3(x2)
        outputs.append(x3)
        # depth 3
        x4 = self.cv4(x3)
        outputs.append(x4)
        # depth 4
        x5 = self.cv5(x4)
        outputs.append(x5)
        # depth 5
        x6 = self.cv6(x5)
        outputs.append(x6)
        
        if d is not None:
            return torch.cat([outputs[i] for i in self.act_idx[:d+1]], dim=1)
        return torch.cat([outputs[i] for i in self.act_idx], dim=1)

In [79]:
input = torch.randn(1, 128, 64, 64)
block = ELANBlock(c1=128, k=3, depth=4)

In [80]:
block(input).shape

torch.Size([1, 320, 64, 64])

5.0